### To run this model yourself: go to Run -> Run All Cells in the top left menu bar.

In [ ]:
from epxexec.fred_job import fred_job

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from epxexec.visual.utils import default_plotly_template
pio.templates["epistemix"] = default_plotly_template()
pio.templates.default = "epistemix"

import plot_mitigations

# Exploring the Utility of Mitigation Strategies for Respiratory Disease

### In this notebook, we will model the spread of an influenza through Kewaunee County, WI.

Our influenza model uses an SEIR structure. If you'd like to learn more about disease transmission in FRED, check out Lesson 7 in the Quickstart Guide.

For now, here's a brief overview of the model:
- We begin with 10 agents randomly exposed to the disease. 
- Exposed agents become Infectious after an average of two days, and they can either be Symptomatic (66%) or Asymptomatic (33%). 
- Agents remain infectious for an average of 5 days, during which they can transmit the disease to other susceptible agents that they come into contact with. 
After the infectious period, they recover and are no longer susceptible to reinfection.

In [ ]:
%%time
job = fred_job("model/main.fred")

## Exploring the Model Output

In [ ]:
baseline_states = plot_mitigations.get_states(job)
baseline_exposures = plot_mitigations.get_explocs(job)
baseline_locations = plot_mitigations.get_expmap_data(job)

In [ ]:
fig = go.Figure()

for x in ["Exposed", "InfectiousA", "InfectiousS", "Recovered"]:
    fig.add_trace(
        go.Scatter(
            x=baseline_states["sim_date"],
            y=baseline_states[x],
            mode="lines",
            line=go.scatter.Line(width=3),
            showlegend=True,
            name=x)
    )

fig.update_layout(
    font_family="Epistemix Label",
    yaxis_title="New infections per day",
    xaxis_title="Date",
    legend_title="State",
    title="The Spread of Influenza in Kewaunee County, WI",
    title_font_size=24,
    xaxis_range=["2022-01-01","2022-04-01"],
    hovermode="x",height=450,
)

fig.show()

Our influenza model tracks how many agents are newly exposed, infectious, or recovered each day. In the figure above, we can see that it takes about two weeks from the initial seeding of exposures at the beginning of the simulation for community spread to increase substantially, and that the disease has run its course after about 10 weeks.

In [ ]:
fig = px.scatter_mapbox(baseline_locations, 
                        lat="my_exp_lat", 
                        lon="my_exp_lon",     
                        color="ExposureLocation",
                        color_discrete_sequence=["#F0438D", "#2BD6AF", "#A76FF4"],
                        opacity=0.8,size="exp_scale",size_max=12,
                        zoom=9.25,height=600,hover_data={'exp_scale':False,
                                                         'NumberExposed':True})


# fig.update_layout(
#     mapbox={'layers': [{
#         'source': blocks_json,
#         'type': "line", 'below': "traces", 'color': "#252E59",
#         'opacity': 1.0
#     }]}
# )

mapstyle="mapbox://styles/pnowell/cl4n9fic8001i15mnfmozrt8j"
token="pk.eyJ1IjoicG5vd2VsbCIsImEiOiJja201bHptMXkwZnQyMnZxcnFveTVhM2tyIn0.Pyarp9gHCON4reKvM2fZZg"

fig.update_layout(mapbox_style=mapstyle, mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":60,"l":0,"b":0})
fig.update_layout(title="Sites of Influenza Exposure",title_font_size=24,)
fig.show()

FRED is a poweful tool not just for understanding disease spread in the aggregate but also for exploring the individual behaviors that are contributing factors. As part of our baseline model, agents recorded where they were exposed to the respiratory disease.

In the map above, we show all the households, workplaces, and schools where agent to agent transmission occurred, with each marker scaled by how many exposures occurred at each location.